In [27]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.tools.retriever import create_retriever_tool #Tool for retrieving data
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain_anthropic import ChatAnthropic
import os
from langchain.agents import create_openai_tools_agent
from langchain import hub #To get the prompt to use
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool #using tool to accomodate any model
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent

In [3]:
'''Toolf for wikipedia'''
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

In [9]:
'''Retriever for obtaining data from the given website'''

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,FastEmbedEmbeddings())
retriever=vectordb.as_retriever()
retriever

/home/user/mambaforge/envs/ecommercesearchai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 73843.38it/s]


VectorStoreRetriever(tags=['FAISS', 'FastEmbedEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f03d416a4a0>)

In [12]:
'''Tool for retrieving data from the given website'''
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

'''name of model :langsmith_search
Description(will be passed to LLM : Search for information about LangSmith. For any questions about LangSmith, you must use this tool! ) '''

'name of model :langsmith_search\nDescription(will be passed to LLM : Search for information about LangSmith. For any questions about LangSmith, you must use this tool! ) '

In [14]:
'''Arxiv Tool'''
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [15]:
'''Combining all the tools'''
tools=[wiki,arxiv,retriever_tool]

In [34]:

os.environ['ANTHROPIC_API_KEY'] = ''
model = ChatAnthropic(model='claude-3-opus-20240229')


In [35]:

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/home/user/mambaforge/envs/ecommercesearchai/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [36]:
'''Binding the tools over the specified model'''
llm_with_tools = model.bind_tools(tools)

In [37]:
'''Agent and Agent executor'''

agent = create_tool_calling_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`
responded: [{'text': '<thinking>\nTo answer the question "Tell me about Langsmith", the most relevant tool is langsmith_search, which searches for information specifically about Langsmith. The query parameter can be inferred as simply "Langsmith" based on the user\'s request.\n\nThe wikipedia and arxiv tools are not relevant here, as they search for general knowledge and scientific papers respectively, while the question is asking for information about the Langsmith company itself.\n\nSince the required query parameter for langsmith_search can be inferred from the user\'s request, I can proceed with making that tool call without needing to ask for any clarification or additional information from the user first.\n</thinking>', 'type': 'text', 'index': 0}, {'id': 'toolu_012G9k52iTzJMLL6Fz9ZTLNn', 'input': {}, 'name': 'langsmith_search', 'type': 'tool_use', 'index': 1, 'partial_json': '{"qu

{'input': 'Tell me about Langsmith',
 'output': [{'text': '\n\n<search_quality_reflection>\nThe langsmith_search results provide a good overview of what Langsmith is and how to get started using it. It explains that Langsmith is a platform for building LLM applications that allows for close monitoring and evaluation. It gives step-by-step instructions for installing Langsmith, creating an API key, setting up your environment, and logging your first trace. \n\nThe results focus mainly on the technical details of getting started, such as the specific commands to run. To fully answer the question of "Tell me about Langsmith", some additional high-level information on Langsmith\'s key features and benefits compared to alternatives would be helpful to include. But overall, the search results are quite relevant and informative for someone looking to learn about Langsmith at an introductory level.\n</search_quality_reflection>\n<search_quality_score>4</search_quality_score>\n\n<result>\nLangs

In [39]:
agent_executor.invoke({"input":"Tell me about a latest paper in Arxiv"})



> Entering new AgentExecutor chain...
[{'text': '<thinking>\nThe arxiv tool seems most relevant to answer this query since it allows searching for the latest scientific articles on arxiv.org in a variety of fields. To use the arxiv tool, I need to provide a search query.\n\nThe user has requested information about a "latest paper in Arxiv", but hasn\'t specified a particular subject area. To get the most relevant latest paper, it would help to narrow down the search to a specific field or topic.\n\nSince the user didn\'t provide a particular area to focus on, I will ask them to clarify what subject they are most interested in hearing about the latest research on. This additional context will allow me to construct a more targeted search query for the arxiv tool.\n</thinking>\n\nTo find the most relevant latest paper on Arxiv for you, it would help to know what subject area or topic you are most interested in. Arxiv covers a wide range of fields including Physics, Mathematics, Computer

{'input': 'Tell me about a latest paper in Arxiv',
 'output': [{'text': '<thinking>\nThe arxiv tool seems most relevant to answer this query since it allows searching for the latest scientific articles on arxiv.org in a variety of fields. To use the arxiv tool, I need to provide a search query.\n\nThe user has requested information about a "latest paper in Arxiv", but hasn\'t specified a particular subject area. To get the most relevant latest paper, it would help to narrow down the search to a specific field or topic.\n\nSince the user didn\'t provide a particular area to focus on, I will ask them to clarify what subject they are most interested in hearing about the latest research on. This additional context will allow me to construct a more targeted search query for the arxiv tool.\n</thinking>\n\nTo find the most relevant latest paper on Arxiv for you, it would help to know what subject area or topic you are most interested in. Arxiv covers a wide range of fields including Physics,